# Dubins-Savage Gambling Strategies

Implementation of optimal gambling strategies from **"How to Gamble If You Must" (Dubins & Savage, 1965)**.

## The Problem

You have initial wealth $w_0$ and want to reach target wealth $W$ before going broke.
Each bet has:
- Probability $p$ of winning
- Odds $b$:1 (win $b$ per unit bet)

**Key Question**: What betting strategy maximizes probability of reaching $W$ before ruin?

## Main Results

For **subfair games** ($p \cdot b < 1-p$, i.e., negative expected value):
- **Bold play is optimal**: Always bet $\min(w, W-w)$
- Minimizes number of bets (and thus exposure to negative edge)

For **superfair games** ($p \cdot b > 1-p$):
- Timid play can be better
- Want more exposure to positive edge

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')

from portfolio_gambling.gambling_strategies import GamblingStrategy

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)

print("Dubins-Savage Gambling Strategies")
print("=" * 50)
print("Based on 'How to Gamble If You Must' (1965)")

## 1. Subfair Game: Casino Roulette

Red/Black bet on American roulette:
- Win probability: p = 18/38 ≈ 0.474
- Odds: 1:1
- Expected value: 18/38 - 20/38 = -2/38 ≈ -5.26%

In [ ]:
# Roulette parameters
initial_wealth = 50
target_wealth = 100
win_prob = 18/38  # Red/Black on American roulette
odds = 1.0  # Even money

game = GamblingStrategy(initial_wealth, target_wealth, win_prob, odds)

print(f"Game Setup:")
print(f"  Initial wealth: ${initial_wealth}")
print(f"  Target wealth: ${target_wealth}")
print(f"  Win probability: {win_prob:.4f}")
print(f"  Expected value per $1 bet: ${game.edge:.4f}")
print(f"  Game type: {game.game_type}")

## 2. Theoretical Probabilities

In [ ]:
# Calculate theoretical probabilities
bold_prob = game.bold_play_probability()
timid_prob = game.timid_play_probability()

print(f"\nTheoretical Success Probabilities:")
print(f"  Bold play:  {bold_prob:.4f} ({bold_prob*100:.2f}%)")
print(f"  Timid play: {timid_prob:.4f} ({timid_prob*100:.2f}%)")
print(f"\n  Bold play is {bold_prob/timid_prob:.2f}x better!")

## 3. Monte Carlo Simulations

In [ ]:
# Run simulations
n_sims = 10000
comparison = game.compare_strategies(n_simulations=n_sims)

print(f"\nMonte Carlo Results ({n_sims} simulations):")
print(f"\nBold Play:")
print(f"  Success rate: {comparison['bold_play']['success_rate']:.4f}")
print(f"  Theoretical:  {comparison['bold_play']['theoretical_prob']:.4f}")
print(f"  Mean # bets:  {comparison['bold_play']['mean_bets']:.1f}")
print(f"  Median # bets: {comparison['bold_play']['median_bets']:.1f}")

print(f"\nTimid Play:")
print(f"  Success rate: {comparison['timid_play']['success_rate']:.4f}")
print(f"  Theoretical:  {comparison['timid_play']['theoretical_prob']:.4f}")
print(f"  Mean # bets:  {comparison['timid_play']['mean_bets']:.1f}")
print(f"  Median # bets: {comparison['timid_play']['median_bets']:.1f}")

## 4. Visualize Final Wealth Distributions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bold play distribution
ax = axes[0]
bold_wealth = comparison['bold_play']['final_wealth_dist']
ax.hist(bold_wealth, bins=50, alpha=0.7, color='blue', edgecolor='black')
ax.axvline(initial_wealth, color='red', linestyle='--', linewidth=2, label=f'Start: ${initial_wealth}')
ax.axvline(target_wealth, color='green', linestyle='--', linewidth=2, label=f'Target: ${target_wealth}')
ax.set_title('Bold Play: Final Wealth Distribution', fontsize=14, fontweight='bold')
ax.set_xlabel('Final Wealth ($)')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(True, alpha=0.3)

# Timid play distribution
ax = axes[1]
timid_wealth = comparison['timid_play']['final_wealth_dist']
ax.hist(timid_wealth, bins=50, alpha=0.7, color='orange', edgecolor='black')
ax.axvline(initial_wealth, color='red', linestyle='--', linewidth=2, label=f'Start: ${initial_wealth}')
ax.axvline(target_wealth, color='green', linestyle='--', linewidth=2, label=f'Target: ${target_wealth}')
ax.set_title('Timid Play: Final Wealth Distribution', fontsize=14, fontweight='bold')
ax.set_xlabel('Final Wealth ($)')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: Bold play is binary (0 or target), timid play has more varied outcomes")

## 5. Effect of Initial Wealth

How does starting wealth affect success probability?

In [ ]:
# Test different starting wealths
target = 100
initial_wealths = np.linspace(10, 90, 20)
bold_probs = []
timid_probs = []

for w0 in initial_wealths:
    g = GamblingStrategy(w0, target, win_prob, odds)
    bold_probs.append(g.bold_play_probability())
    timid_probs.append(g.timid_play_probability())

# Plot
plt.figure(figsize=(12, 7))
plt.plot(initial_wealths, bold_probs, 'b-', linewidth=3, label='Bold Play', marker='o')
plt.plot(initial_wealths, timid_probs, 'orange', linewidth=3, label='Timid Play', marker='s')
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='50% chance')
plt.xlabel('Initial Wealth ($)', fontsize=12)
plt.ylabel('Probability of Reaching $100', fontsize=12)
plt.title('Success Probability vs Starting Wealth (Subfair Game)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\nAs you get closer to the target, success probability increases for both strategies")
print("But bold play ALWAYS dominates in subfair games!")

## 6. Superfair Game Example

What if you have an edge? (e.g., counting cards in blackjack)

In [ ]:
# Superfair game: slight edge
superfair_game = GamblingStrategy(
    initial_wealth=50,
    target_wealth=100,
    win_prob=0.51,  # Slight edge
    win_odds=1.0
)

print(f"Superfair Game:")
print(f"  Win probability: {superfair_game.p:.4f}")
print(f"  Expected value per $1: ${superfair_game.edge:.4f}")
print(f"  Game type: {superfair_game.game_type}")

# Compare strategies
superfair_comp = superfair_game.compare_strategies(n_simulations=10000)

print(f"\nSuccess Probabilities (with edge):")
print(f"  Bold play:  {superfair_comp['bold_play']['success_rate']:.4f}")
print(f"  Timid play: {superfair_comp['timid_play']['success_rate']:.4f}")

if superfair_comp['timid_play']['success_rate'] > superfair_comp['bold_play']['success_rate']:
    print(f"\n  Timid play wins! (More bets = more edge exploitation)")
else:
    print(f"\n  Bold play still wins (edge too small)")

## 7. The "Double Your Money" Problem

Classic question: You have $50, want $100, play roulette. What's your best strategy?

In [ ]:
scenarios = [
    ("Bet everything once", 50, 1),
    ("Bet $25 twice", 25, 2),
    ("Bet $10 five times", 10, 5),
    ("Bet $5 ten times", 5, 10),
    ("Bet $1 fifty times", 1, 50),
]

print("\nDouble Your Money Strategies:")
print("=" * 60)

for strategy, bet_size, n_bets in scenarios:
    # Rough approximation: need to win enough to reach $100
    # This is simplified - exact calculation requires dynamic programming
    
    if n_bets == 1:
        prob = win_prob  # Win once
    else:
        # Approximate - not exact for all strategies
        needed_net_wins = (target_wealth - initial_wealth) / bet_size
        
        # Simplified: assume we need to win at least this many MORE than we lose
        # Real calculation is more complex
        prob = game.timid_play_probability(bet_size=bet_size)
    
    print(f"{strategy:30s}: {prob:.4f} ({prob*100:.2f}%)")

print("\n" + "="*60)
print("Bold play (bet everything) is optimal!")
print("Each additional bet subjects you to the house edge again.")

## 8. Connection to Kelly Criterion

**Dubins-Savage**: Maximize P(reach target before ruin)
- Finite horizon
- Binary outcome (success/failure)
- Subfair games → bold play

**Kelly Criterion**: Maximize E[log(wealth)]
- Infinite horizon
- Continuous growth
- Even with edge → fractional betting (not bold play!)

### Example: Kelly vs Bold Play with Edge

In [ ]:
# Game with edge
p = 0.55  # 55% win probability
b = 1.0   # Even odds

# Kelly fraction: f* = (p*b - q) / b = (0.55*1 - 0.45) / 1 = 0.10
kelly_fraction = (p * b - (1-p)) / b

print(f"\nGame with Edge:")
print(f"  Win probability: {p}")
print(f"  Expected value: {p * b - (1-p):.3f} per dollar")
print(f"\nOptimal Strategies:")
print(f"  Kelly fraction: {kelly_fraction:.3f} (bet {kelly_fraction*100:.1f}% of bankroll)")
print(f"  Dubins-Savage: Depends on target!")
print(f"    - If target is nearby: bold play still good")
print(f"    - If target is far: timid play may be better")
print(f"\nKey difference: Kelly maximizes long-run growth, D-S maximizes P(reach specific target)")

## 9. Practical Implications

### When to use Bold Play:
1. Playing against the house (negative expectation)
2. Have a specific wealth target in mind
3. Want to maximize probability of reaching that target
4. Limited time/opportunities

### When NOT to use Bold Play:
1. You have an edge (positive expectation)
2. Long-term wealth accumulation is the goal
3. No specific target - just want to grow wealth
4. Risk management is important (bold play has high volatility)

### Real-world Applications:
- **Venture capital**: Sometimes bold bets on moonshots (subfair but specific target)
- **Tournament poker**: Bold play near bubble (maximize probability of cashing)
- **Options strategies**: Buying lottery-ticket calls (subfair, but targeting specific gain)
- **Portfolio management**: Usually Kelly-style (long-run growth, no fixed target)

## Key Takeaways

1. **Subfair games**: Bold play is mathematically optimal for reaching a target

2. **Minimizing exposure**: Bold play minimizes number of bets, reducing exposure to house edge

3. **All-or-nothing**: Bold play leads to binary outcomes (ruin or success)

4. **Different from Kelly**: D-S optimizes finite-horizon target probability, Kelly optimizes infinite-horizon growth

5. **Practical wisdom**: "If you must gamble (subfair), go big or go home!"

6. **Portfolio analog**: Sometimes bold concentrated bets beat diversification (when forced into negative-expectation situation)